# Guide to using Trino

In this example notebook:

* "SAIL" is the name of the remote TRE

* "EPCC" is the name of the hosting TRE

## Connecting

In [ ]:
import os

# Your Trino username goes here
TRINO_USERNAME = ""
# Your Trino password goes here
TRINO_PASS = ""
# no http, e.g. trino.teleport.pk.serp.ac.uk
TRINO_URL = "trino.teleport.pk.serp.ac.uk"

In [3]:
import pandas as pd
from trino.auth import BasicAuthentication
from sqlalchemy import create_engine, text

engine = create_engine(
    "trino://{0}:{1}@{2}:443".format(TRINO_USERNAME, TRINO_PASS, TRINO_URL),
    connect_args={
        "auth": BasicAuthentication(TRINO_USERNAME, TRINO_PASS),
        "http_scheme": "https",
        # You may need to uncomment out the line below if you are getting certificate errors         
        # "verify": False
    }
)

def call(q):
    ResultProxy = engine.connect().execute(text(q))
    results = ResultProxy.fetchall()
    dataframe = pd.DataFrame(results)
    dataframe.columns = ResultProxy.keys()
    return dataframe

## Basic querying

### Make sure the Trino connection is working

In [4]:
q = 'SELECT * FROM system.runtime.nodes LIMIT 3'
call(q)

,node_id,http_uri,node_version,coordinator,state
0,7d804d793929,http://172.26.0.4:8080,420,True,active


### See connectors, schemas, and tables available to you

In [5]:
# See connectors
q = 'SHOW catalogs'
display(call(q))

# # See schemas in connector/catalog, assuming you have a connector called "epcc"
q = 'SHOW schemas in epcc'
display(call(q))

# # See tables in a schema, assuming connecter called "epcc", schema called "sf300"
q = 'SHOW tables in epcc.sf300'
display(call(q))

,Catalog
0,epcc
1,sail
2,system


,Schema
0,information_schema
1,sf1
2,sf10
3,sf100
4,sf1000
5,sf10000
6,sf100000
7,sf300
8,sf3000
9,sf30000


,Table
0,call_center
1,catalog_page
2,catalog_returns
3,catalog_sales
4,customer
5,customer_address
6,customer_demographics
7,date_dim
8,dbgen_version
9,household_demographics


### Query a table in your hosting TRE

In [6]:
# replace "tablename" with the name of the table
# assuming you're using the connector called "epcc" and table is in a schema called "sf300"
q = 'SELECT * FROM epcc.sf300.warehouse LIMIT 5'
# read the result of this query into a dataframe
df = call(q)
df

,w_warehouse_sk,w_warehouse_id,w_warehouse_name,w_warehouse_sq_ft,w_street_number,w_street_name,w_street_type,w_suite_number,w_city,w_county,w_state,w_zip,w_country,w_gmt_offset
0,1,AAAAAAAABAAAAAAA,Conventional childr,977787.0,651,6th,Parkway,Suite 470,Pleasant Hill,San Miguel County,NM,83604,United States,-7.00
1,2,AAAAAAAACAAAAAAA,Important issues liv,138504.0,600,View First,Avenue,Suite P,Salem,Huron County,MI,48048,United States,-5.00
2,3,AAAAAAAADAAAAAAA,Doors canno,294242.0,534,Ash Laurel,Dr.,Suite 0,Salem,Ziebach County,SD,58048,United States,-6.00
3,4,AAAAAAAAEAAAAAAA,Bad cards must make.,621234.0,368,Wilson Elm,Drive,Suite 80,Salem,Richland County,OH,48048,United States,-5.00
4,5,AAAAAAAAFAAAAAAA,None,NaN,None,None,None,None,Five Points,Luce County,MI,46098,United States,None


### Understand what catalogs, schemas, and tables are available in the remote TRE
By using **query passthrough** to the `system.jdbc` schema, you can get a list of things available in the remote TRE

#### Catalogs

In [7]:
q = '''
SELECT * FROM TABLE(
    sail.system.query(
        query => '
                SELECT * FROM system.jdbc.catalogs
                '
    )
)'''
call(q)

,table_cat
0,jmx
1,memory
2,system
3,tpcds
4,tpch


#### Schemas

In [8]:
q = '''
SELECT * FROM TABLE(
    sail.system.query(
        query => '
                SELECT * FROM system.jdbc.schemas
                '
    )
)'''
call(q)

,table_schem,table_catalog
0,current,jmx
1,history,jmx
2,information_schema,jmx
3,default,memory
4,information_schema,memory
5,information_schema,system
6,jdbc,system
7,metadata,system
8,runtime,system
9,information_schema,tpcds


#### Tables

In [9]:
q = '''
SELECT * FROM TABLE(
    sail.system.query(
        query => '
                SELECT * FROM system.jdbc.tables
                '
    )
)'''
call(q)

,table_cat,table_schem,table_name,table_type,remarks,type_cat,type_schem,type_name,self_referencing_col_name,ref_generation
0,jmx,information_schema,columns,TABLE,None,None,None,None,None,None
1,jmx,information_schema,tables,TABLE,None,None,None,None,None,None
2,jmx,information_schema,views,TABLE,None,None,None,None,None,None
3,jmx,information_schema,schemata,TABLE,None,None,None,None,None,None
4,jmx,information_schema,table_privileges,TABLE,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
724,tpch,sf100000,part,TABLE,None,None,None,None,None,None
725,tpch,sf100000,partsupp,TABLE,None,None,None,None,None,None
726,tpch,sf100000,supplier,TABLE,None,None,None,None,None,None
727,tpch,sf100000,nation,TABLE,None,None,None,None,None,None


If you want to filter by a text value with query passthrough, it's important that you put the string value inside double apostrophes.

In [25]:
q = '''
SELECT * FROM TABLE(
    sail.system.query(
        query => '
                SELECT * FROM system.jdbc.tables WHERE table_cat = ''tpch''
                '
    )
)'''
call(q)

,table_cat,table_schem,table_name,table_type,remarks,type_cat,type_schem,type_name,self_referencing_col_name,ref_generation
0,tpch,information_schema,columns,TABLE,None,None,None,None,None,None
1,tpch,information_schema,tables,TABLE,None,None,None,None,None,None
2,tpch,information_schema,views,TABLE,None,None,None,None,None,None
3,tpch,information_schema,schemata,TABLE,None,None,None,None,None,None
4,tpch,information_schema,table_privileges,TABLE,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
75,tpch,sf100000,part,TABLE,None,None,None,None,None,None
76,tpch,sf100000,partsupp,TABLE,None,None,None,None,None,None
77,tpch,sf100000,supplier,TABLE,None,None,None,None,None,None
78,tpch,sf100000,nation,TABLE,None,None,None,None,None,None


### Query a table in the remote TRE
When you query a table in the remote TRE, you should always use **query passthrough**. 

This will make your queries much faster.

Here is an example of how to use query passthrough to get a subset of data from the 'part' table in the 'sf1' schema using the 'tpch' connector.
We want only rows where `size` is greater than or equal to 50.

In [32]:
q = '''
SELECT * FROM TABLE(
    sail.system.query(
        query => '
                SELECT * FROM tpch.sf1.part 
                WHERE size >= 50
                '
    )
)'''
result = call(q)
display(result)

,partkey,name,mfgr,brand,type,size,container,retailprice,comment
0,150005,firebrick drab puff grey lawn,Manufacturer#5,Brand#51,LARGE BURNISHED COPPER,50,JUMBO BOX,1055.00,nts haggle
1,150116,yellow almond magenta smoke cornsilk,Manufacturer#2,Brand#23,MEDIUM BRUSHED NICKEL,50,JUMBO CASE,1166.11,"es. even, final p"
2,150174,khaki cornsilk magenta dodger turquoise,Manufacturer#4,Brand#42,ECONOMY BURNISHED BRASS,50,LG PACK,1224.17,fully reg
3,150177,tan red blue chocolate powder,Manufacturer#1,Brand#15,STANDARD ANODIZED COPPER,50,SM DRUM,1227.17,nts wake
4,150224,goldenrod puff ivory black olive,Manufacturer#1,Brand#13,ECONOMY BRUSHED STEEL,50,WRAP CASE,1274.22,w around the de
...,...,...,...,...,...,...,...,...,...
3884,149684,linen lavender grey brown lemon,Manufacturer#4,Brand#45,SMALL PLATED COPPER,50,JUMBO BAG,1733.68,fully final pint
3885,149748,lemon bisque dim chartreuse frosted,Manufacturer#3,Brand#33,LARGE BURNISHED COPPER,50,LG PKG,1797.74,lose quick
3886,149768,cream ghost bisque burlywood thistle,Manufacturer#1,Brand#15,PROMO BRUSHED BRASS,50,SM JAR,1817.76,le carefully slyly
3887,149840,midnight medium sienna metallic saddle,Manufacturer#3,Brand#32,LARGE BRUSHED COPPER,50,JUMBO CASE,1889.84,nic dolphins engag


In [33]:
q = '''
SELECT * FROM TABLE(
    sail.system.query(
        query => '
                SELECT COUNT(*) as num FROM tpch.sf1.part
                '
    )
)'''
result = call(q)
display(result)

,num
0,200000


We can see that the filtered query returned 3889 rows, with the entire table having 200,000 rows. 
All of the query filtering was carried out on servers at the remote TRE, so *only* the data from the 3889 rows had to be transported
across the internet to the hosting TRE.

If we did this query without passthrough:

In [34]:
q = '''
SELECT * FROM sail.sf1.part
WHERE size >= 50
'''

The remote TRE would have to send all 200k rows to the hosting TRE before any of the filtering could be done. 

Those sort of queries will take a very long time to run.